In [ ]:
import numpy as np
from matplotlib import pyplot
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from path_guiding import VMFMixture8, VMFFitIncremental8

In [ ]:
%matplotlib notebook

In [ ]:
def plot_vmf_pdf(ax, vmfm):
    # https://scipython.com/book/chapter-8-scipy/examples/visualizing-the-spherical-harmonics/
    phi = np.linspace(0, np.pi, 100)
    theta = np.linspace(0, 2*np.pi, 100)
    phi, theta = np.meshgrid(phi, theta)

    # The Cartesian coordinates of the unit sphere
    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)

    pts = np.vstack((x.ravel(),y.ravel(),z.ravel())).T
    fcolors = vmfm.pdf(pts).reshape(x.shape)
    fcolors /= fcolors.max()
    
    ax.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=matplotlib.cm.coolwarm(fcolors), shade=False)
    return fcolors

In [ ]:
VMFMixture = VMFMixture8

def two_modes_vmfm():
    vmfm = VMFMixture()
    w = vmfm.weights 
    w[:] = 0
    w[0] = 0.2
    w[1] = 0.8
    vmfm.weights = w
    c = vmfm.concentrations
    c[:] = 20
    vmfm.concentrations = c
    m = vmfm.means
    m[:,:] = 0
    m[0,:] = [1, 0, 0]
    m[1,:] = [0, 1, 0]
    vmfm.means = m
    return vmfm

def overlapping_peaks_vmfm():
    vmfm = VMFMixture()
    w = vmfm.weights 
    w[:] = 0
    w[0] = 0.5 #0.05
    w[1] = 0.5 #0.95
    vmfm.weights = w
    c = vmfm.concentrations
    c[:] = 1
    c[0] = 40.
    c[1] = 2.
    vmfm.concentrations = c
    m = vmfm.means
    m[:,:] = 0
    m[0,:] = [1, 0, 0]
    m[1,:] = [1, 0, 0]
    vmfm.means = m
    return vmfm

def three_modes_vmfm():
    vmfm = VMFMixture()
    w = vmfm.weights 
    w[:] = 0
    w[0] = 0.2 
    w[1] = 0.6
    w[2] = 0.2
    vmfm.weights = w
    c = vmfm.concentrations
    c[:] = 0.1
    c[0] = 5.
    c[1] = 1.
    c[2] = 5.
    vmfm.concentrations = c
    m = vmfm.means
    m[:,:] = 0
    m[0,:] = [1, 0, 0]
    m[1,:] = [0, 0, 1]
    m[2,:] = [-1, 0, 0]
    vmfm.means = m
    return vmfm

def uniform_vmfm():
    vmfm = VMFMixture()
    w = vmfm.weights 
    w[:] = 0
    w[0] = 1.
    vmfm.weights = w
    c = vmfm.concentrations
    c[:] = 0.1
    vmfm.concentrations = c
    return vmfm

def make_half_sphere_vmfm():
    vmfm = VMFMixture()
    means = vmfm.means
    i = means[:,0] < 0.
    w = vmfm.weights
    w[i] = 0.
    w[~i] *= 2.
    vmfm.weights = w
    return vmfm

def _shuffle(xs, ws):
    n = xs.shape[0]
    idx = np.random.permutation(n)
    return xs[idx,...], ws[idx]


def _make_samples_decorator(vmfm_fun):
    def wrapper(n):
        vmfm = vmfm_fun()
        samples = vmfm.sample(n)    
        return _shuffle(samples, np.ones(n, np.float32))
    return wrapper

def _make_samples_weighted_decorator(vmfm_fun):
    def wrapper(n):
        vmfm = vmfm_fun()
        samples = uniform_vmfm().sample(n)
        weights = vmfm.pdf(samples)
        return _shuffle(samples, weights)
    return wrapper


make_uniform_samples = _make_samples_decorator(uniform_vmfm)
make_two_modes_samples = _make_samples_decorator(two_modes_vmfm)
make_half_sphere_samples = _make_samples_decorator(make_half_sphere_vmfm)
make_three_modes_samples = _make_samples_decorator(three_modes_vmfm)
make_overlapping_peaks_samples = _make_samples_decorator(overlapping_peaks_vmfm)

make_two_modes_weighted = _make_samples_weighted_decorator(two_modes_vmfm)
make_three_modes_weighted = _make_samples_weighted_decorator(three_modes_vmfm)
make_overlapping_peaks_weighted = _make_samples_weighted_decorator(overlapping_peaks_vmfm)

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
plot_vmf_pdf(ax, overlapping_peaks_vmfm())
pyplot.show()

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')

samples, weights = make_overlapping_peaks_samples(1000)

ax.scatter(*samples.T, c=weights)

In [ ]:
prior_nu = 1
prior_alpha = 1
prior_tau = 1
maximization_step_every = 100;
prior_mode = VMFMixture()
incremental = VMFFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

xs, ws = make_uniform_samples(1000)

vmfm = VMFMixture()
incremental.fit(vmfm, xs, ws)

fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
v = plot_vmf_pdf(ax, vmfm)

m = matplotlib.cm.ScalarMappable(cmap=matplotlib.cm.coolwarm)
m.set_array(v)
pyplot.colorbar(m)

pyplot.show()

In [ ]:
prior_nu = 100.
prior_alpha = 100.
prior_tau = 100.
maximization_step_every = 100;
prior_mode = VMFMixture()
incremental = VMFFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

xs, ws = make_half_sphere_samples(1000)
vmfm = VMFMixture()
incremental.fit(vmfm, xs, ws)

fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
v = plot_vmf_pdf(ax, vmfm)

print (vmfm.concentrations)

m = matplotlib.cm.ScalarMappable(cmap=matplotlib.cm.coolwarm)
m.set_array(v)
pyplot.colorbar(m)

pyplot.show()

In [ ]:
prior_nu = 100.
prior_alpha = 100.
prior_tau = 100.
maximization_step_every = 100;
prior_mode = VMFMixture()
incremental = VMFFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

xs, ws = make_two_modes_samples(1000)
vmfm = VMFMixture()
incremental.fit(vmfm, xs, ws)

fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
plot_vmf_pdf(ax, vmfm)
pyplot.show()

In [ ]:
prior_nu = 1.
prior_alpha = 1.
prior_tau = 1.
maximization_step_every = 100;
prior_mode = VMFMixture()
incremental = VMFFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

xs, ws = make_two_modes_weighted(1000)
vmfm = VMFMixture()
incremental.fit(vmfm, xs, ws)

fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
plot_vmf_pdf(ax, vmfm)
ax.scatter(*(vmfm.means*1.1).T, marker='x', c='r', s = 30.)
pyplot.show()

In [ ]:
prior_nu = 1.
prior_alpha = 1.
prior_tau = 1.
maximization_step_every = 100;
prior_mode = VMFMixture()
incremental = VMFFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

xs, ws = make_three_modes_samples(1000)
vmfm = VMFMixture()
incremental.fit(vmfm, xs, ws)

fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
plot_vmf_pdf(ax, vmfm)
ax.scatter(*(vmfm.means*1.1).T, marker='x', c='r', s = 30.)
pyplot.show()

In [ ]:
prior_nu = 1.
prior_alpha = 1.
prior_tau = 1.
maximization_step_every = 100;
prior_mode = VMFMixture()
incremental = VMFFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

xs, ws = make_three_modes_weighted(1000)
vmfm = VMFMixture()
incremental.fit(vmfm, xs, ws)

fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
plot_vmf_pdf(ax, vmfm)
ax.scatter(*(vmfm.means*1.1).T, marker='x', c='r', s = 30.)
pyplot.show()

In [ ]:
prior_nu = 1.
prior_alpha = 1.
prior_tau = 1.
maximization_step_every = 100;
prior_mode = VMFMixture()
incremental = VMFFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

#xs, ws = make_overlapping_peaks_samples(1000)
xs, ws = make_overlapping_peaks_weighted(1000)
vmfm = VMFMixture()
incremental.fit(vmfm, xs, ws)

print (vmfm.weights)

fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
plot_vmf_pdf(ax, vmfm)
ax.scatter(*(vmfm.means*1.1).T, marker='x', c='r', s = 30.)
pyplot.show()